In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
from skimage.io import imread
import torch.nn.functional as F
%matplotlib qt5

In [2]:
def get_dots_wavelet(image, noise_threshold, segmentation_mask,
                    min_spot_area, max_axes_ratio):
    """
    A function that outputs (x, y) coordinates of the dots in an image
    Args:
        image (np.ndarray): an image 
        noise_threshold:
        segmentation_mask:
        min_spot_area:
        max_axes_ratio:
        

    Returns:
        dot_coords(np.ndarray): a numpy array of shape (N x 2) where n is the number
                    of dots in the image, and each row is (x_coordinate, y_coordinate)
    """
    # detect spots

    # find weighted centroid of the spots
    
    pass

In [3]:
def wavelet_spot_dectect(image):
    """
    

    Returns:
        binary_mask (np.ndarray or torch.Tensor): 
    """
    pass

In [ ]:
def ATrousTransform(image, levels):
    pass

#### Hard coded constants

In [5]:
waveletPlanes = 2
levels = 3

In [6]:
fluor_image_path = Path('/home/pk/Documents/waveletCode/data/img_000000000.tiff')

In [7]:
fluor_img = imread(fluor_image_path).astype('float32')

In [8]:
plt.figure()
plt.imshow(fluor_img, cmap='gray')
plt.show()

In [9]:
fluor_img.shape, fluor_img.dtype

((1404, 3200), dtype('float32'))

In [10]:
plt.figure()
plt.imshow(2*np.sqrt(fluor_img+3/8))
plt.show()

In [11]:
fluor_img_tensor = torch.from_numpy(fluor_img)
anscombe_trans_tensor = 2 * torch.sqrt(fluor_img_tensor + 3/8)

In [12]:
anscombe_trans_tensor = 2 * torch.sqrt(fluor_img_tensor + 3/8)

In [13]:
anscombe_trans_image = 2 * np.sqrt(fluor_img+3/8)

In [14]:
anscombe_trans_tensor.device

device(type='cpu')

### A trous transform

In [15]:
def atrous_transform(anscombe_image, levels: int = 3, device='cpu'):
    """
    
    Args:

    Returns:
        
    """
    H, W = anscombe_image.shape
    
    #print(f"Anscombe image shape: {H}x{W}")
    wavelet_planes = torch.zeros((levels+1, H, W), device=device)
    base_kernel1 = (1/16) * torch.tensor([1, 4, 6, 4, 1], device=device)
    base_kernel2 = (1/16) * torch.tensor([1, 0, 4, 0, 6, 0, 4, 0, 1], device=device)
    base_kernel3 = (1/16) * torch.tensor([1, 0, 0, 0, 4, 0, 0, 0, 6, 0, 0, 0, 4, 0, 0, 0, 1],
                                        device=device)
    
    kernel1 = (base_kernel1[:, None] * base_kernel1[None, :])[None, None, :]
    kernel2 = (base_kernel2[:, None] * base_kernel2[None, :])[None, None, :]
    kernel3 = (base_kernel3[:, None] * base_kernel3[None, :])[None, None, :]
    #kernel1.to(device)
    #kernel2.to(device)
    #kernel3.to(device)

    input_tensor = anscombe_image[None, None,:].to(device)
    last_tensor = torch.Tensor(input_tensor)
    
    out_tensor1 = F.conv2d(last_tensor, kernel1, padding='same')
    wavelet_planes[0] = (input_tensor - out_tensor1)[:, :]
    last_tensor = out_tensor1
    out_tensor2 = F.conv2d(last_tensor, kernel2, padding='same')
    wavelet_planes[1] = (last_tensor - out_tensor2)[:, :]
    last_tensor = out_tensor2
    out_tensor3 = F.conv2d(last_tensor, kernel3, padding='same')
    wavelet_planes[2] = (last_tensor - out_tensor3)[:, :]
    last_tensor = out_tensor3
    wavelet_planes[3] = last_tensor

    return wavelet_planes

In [16]:
w = atrous_transform(anscombe_trans_tensor, device='cpu')

In [17]:
%timeit w = atrous_transform(anscombe_trans_tensor, device='cpu')

2.02 s ± 228 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
w_gpu = atrous_transform(anscombe_trans_tensor, device='cuda:0')

In [19]:
%timeit w_gpu = atrous_transform(anscombe_trans_tensor, device='cuda:0')

3.15 ms ± 96.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
w.device, w_gpu.device

(device(type='cpu'), device(type='cuda', index=0))

In [21]:
torch.max(torch.abs((w - w_gpu.cpu()))) 

tensor(1.9073e-05)

In [22]:
def plot_wavelets(wavelet_planes):
    fig, ax = plt.subplots(nrows=2, ncols=2)
    for i in range(4):
        ax[i//2, i % 2].imshow(wavelet_planes[i].numpy())
        ax[i//2, i % 2].set_title(f"i = {i}th plane")
    plt.show()

In [23]:
plot_wavelets(w_gpu.cpu())

In [24]:
plot_wavelets(w)

#### Construction binary mask of the spots using the wavelet planes

In [25]:
def filter_image():
    pass

In [ ]:
def construct_binary_spot_mask():
    pass